# Converting from Qiskit Runtime Programs

This tutorial will be a demonstation of converting your custom Qiskit Runtime Program into a Quantum Serverless `Program`.

If you were using Qiskit Runtime Programs before, your code probably looks similar to the following example:

```python
"""A sample runtime program that submits random circuits for user-specified iterations."""

import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit


def prepare_circuits(backend):
    circuit = random_circuit(
        num_qubits=5, depth=4, measure=True, seed=random.randint(0, 1000)
    )
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop("iterations", 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    return "Hello, World!"
```


All Qiskit Runtime Programs have a `main` method which accepts `backend`, `user_messenger` and `**kwargs`. This method is not required for Quantum Serverless programs.

Quantum Serverless handles backends, logging, and input arguments a bit differently than Qiskit Runtime:

- `backend`. For Quantum Serverless programs you are not limited to single backend for a program. You can call any
  number of backends from single program. Since `Backend.run` is deprecated, we will be using Qiskit Primitives to do our calculation.
- `user_messenger` were used in Qiskit Runtime Programs to facilitate retrieving logs from the program. Quantum Serverless does not
  require passing such an object. Instead, all contents of `stdout` (e.g. print statements, logging messages) will be provided to the
  user via the Quantum Serverless job handler.
- `**kwargs` was a variable used to capture program inputs from the user. Users should now input their arguments to the `Program` constructor,
  and the arguments should be retrieved within the program using the `get_arguments` function from Quantum Serverless.
- To save the results of a program, the `save_result` function should be used. It accepts a python dictionary and can be accessed via the job handler.

Let's use the guidelines above to transform the above Qiskit Runtime Program into a Quantum Serverless Program.

```python
# migrated_program.py
"""A sample runtime program that submits random circuits for user-specified iterations."""

import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Sampler

from quantum_serverless import get_arguments, save_result


def prepare_circuits():
    circuit = random_circuit(
        num_qubits=5, depth=4, measure=True, seed=random.randint(0, 1000)
    )
    return transpile(circuit)


arguments = get_arguments()
iterations = arguments.get("iterations", 5)

for it in range(iterations):
    qc = prepare_circuits()
    result = Sampler.run(qc).result()
    print({"iteration": it, "dists": result.quasi_dists})

save_result({"result": "Hello, World!"})
```

Let's save this code as `./src/migrated_program.py` and execute it using the `Program` class from the `quantum_serverless` package.

In [1]:
from quantum_serverless import Program

program = Program(
    title="Migrated program", entrypoint="migrated_program.py", working_dir="./src/"
)

In [2]:
from quantum_serverless import QuantumServerless, ServerlessProvider
import os

provider = ServerlessProvider(
    username=os.environ.get("GATEWAY_USER", "user"),
    password=os.environ.get("GATEWAY_PASSWORD", "password123"),
    # token=os.environ.get("GATEWAY_TOKEN", "<TOKEN>"), # token can be used instead of user/password combination
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

While Qiskit Runtime programs required users to upload their program and call it in two separate steps, the ``Program`` class allows users to send a job for remote execution in a single step.

In [19]:
job = serverless.run(program, arguments={"iterations": 3})
job

<Job | 9c5010df-e2f2-42d2-ac44-be5d28bf6741>

In [20]:
job.result()

'{"result": "Hello, World!"}'

In [21]:
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
{'iteration': 0, 'dists': [{8: 0.0044464220610969, 9: 0.0714788704988803, 10: 0.0025387094251744, 11: 0.0408112589724704, 12: 0.0044464220610969, 13: 0.0714788704988803, 14: 0.0025387094251744, 15: 0.0408112589724704, 24: 0.0323482396235251, 25: 0.2100040886340835, 26: 0.018469407467762, 27: 0.1199030033170077, 28: 0.0323482396235251, 29: 0.2100040886340835, 30: 0.018469407467762, 31: 0.1199030033170077}]}
{'iteration': 1, 'dists': [{0: 0.0029658828912998, 1: 0.0237036789902499, 4: 0.0150644475889747, 5: 0.1203968069211296, 8: 0.0061806907129902, 9: 0.0493967948055204, 10: 0.0035529476369869, 11: 0.0283955683157122, 12: 0.0313932460322799, 13: 0.2508984520575557, 14: 0.0144443775915554, 15: 0.1154411358076723, 16: 2.64895095504e-05, 17: 0.0002117072231119, 20: 6.5157421817e-06, 21: 5.20745648832e-05, 24: 5.52022691